In [2]:
!pip install azure-ai-ml

     |████████████████████████████████| 7.0 MB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 123 kB 82.8 MB/s eta 0:00:01
     |████████████████████████████████| 266 kB 84.9 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 4.6 MB/s  eta 0:00:01
     |████████████████████████████████| 249 kB 88.3 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 7.7 MB/s  eta 0:00:01
ERROR: azure-storage-file-share 12.14.1 has requirement azure-core<2.0.0,>=1.28.0, but you'll have azure-core 1.26.4 which is incompatible.
ERROR: azure-storage-file-datalake 12.13.1 has requirement azure-core<2.0.0,>=1.28.0, but you'll have azure-core 1.26.4 which is incompatible.
ERROR: azure-storage-file-datalake 12.13.1 has requirement azure-storage-blob<13.0.0,>=12.18.1, but you'll have azure-storage-blob 12.13.0 which is incompatible.


### Connect to your workspace

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


### Create a compute instance programmatically using python SDK

In [14]:
from azure.ai.ml.entities import ComputeInstance

compute_instance_name="coinstancev2010"

try:
    compute_instance = ml_client.compute.get(compute_instance_name)
    print(f"Using an existing instance of the {compute_instance_name} compute instance..")

except:
    print(f"Compute resource: {compute_instance_name} does not exist so will need to create it. This will take a couple of minutes..")
    compute_instance = ComputeInstance(
        name=compute_instance_name,
        size="STANDARD_DS3_V2",
    )
    returned_job = ml_client.compute.begin_create_or_update(compute_instance)
    print(f"Job response: {returned_job.studio_url}")


Using an existing instance of the coinstancev2010 compute instance


### Create a script to train a model

In [2]:
%%writefile Src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

Writing Src/diabetes-training.py


### Run a job on the compute instance

In [8]:
from azure.ai.ml import command

job_compute_instance = command(
    code="Src",
    command="python diabetes-training.py",
    compute="coinstancev2010",
    experiment_name="diabetes-training",
    display_name="Job using compute instance",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    )

returned_job = ml_client.create_or_update(job_compute_instance)
print(f"Job response: {returned_job.studio_url}")

Uploading Src (0.52 MBs): 100%|██████████| 519221/519221 [00:00<00:00, 2360437.93it/s]




Job response: https://ml.azure.com/runs/sweet_spring_1wy5xvj5wl?wsid=/subscriptions/16f263d1-840b-4414-8415-243b4662d595/resourcegroups/rg-dp100-labs/workspaces/mlw-dp100-labs-v1010&tid=539aca29-18ae-4a65-9554-5e301a4618ce


### Create a compute cluster

In [11]:
from azure.ai.ml.entities import AmlCompute

compute_cluster_name = "aml-compute"

try:
    cpu_cluster = ml_client.compute.get(name=compute_cluster_name)

except:
    print(f"Compute resource: {compute_cluster_name} does not exist so will need to create it. This will take a couple of minutes..")
    cpu_cluster = AmlCompute(
        name=compute_cluster_name,
        size="STANDARD_DS11_V2",
        min_instances=0,
        max_instances=4,
        type="amlcompute",
        idle_time_before_scale_down=120,
        tier="dedicated",
    )

    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)


Compute resource: aml-compute does not exist so will need to create it. This will take a couple of minutes..


### Run a job on the compute cluster

In [12]:
job_compute_cluster = command(
    code="./Src",
    command="python diabetes-training.py",
    compute=compute_cluster_name,
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    experiment_name="diabetes-training",
    display_name="Job using compute cluster",
)

returned_job = ml_client.create_or_update(job_compute_cluster)
print(f"Job response: {returned_job.studio_url}")

Job response: https://ml.azure.com/runs/honest_kumquat_m4m7d77q2x?wsid=/subscriptions/16f263d1-840b-4414-8415-243b4662d595/resourcegroups/rg-dp100-labs/workspaces/mlw-dp100-labs-v1010&tid=539aca29-18ae-4a65-9554-5e301a4618ce
